In [1]:
import findspark
findspark.init('/opt/spark')  # Adjust the path if Spark is installed elsewhere

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("example").getOrCreate()

24/06/11 23:11:15 WARN Utils: Your hostname, chuot-HP-Pavilion-Laptop-14-ce3xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.14 instead (on interface wlo1)
24/06/11 23:11:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/11 23:11:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/11 23:11:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/11 23:11:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/06/11 23:11:16 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
# Define the path to your JSON file
from pyspark.sql.functions import col, udf, to_date, when, regexp_extract, explode, col, regexp_replace, to_timestamp, concat, lit, array, trim
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType, ArrayType
import json
import re
import pymongo
json_file_path = 'vieclam24h.json'
# Read the JSON file into a DataFrame
df = spark.read.option("multiline","true").json(json_file_path)

df_exploded = df.select(explode(col("location")).alias("unique_category"))

unique_categories = df_exploded.select("unique_category").distinct().collect()
print(len(unique_categories))
for row in unique_categories:
    pass
    print(row.unique_category)
unique_experience_df = df.select("location").distinct()

unique_experience_df.show()

# from pyspark.sql.functions import col
# unique_values = df.select(col("experience")).distinct().collect()

# # In ra các giá trị duy nhất
# for row in unique_values:
#     if row.experience is None:
#         print("None ne")
#     print(row.experience)


# print(df.size)


#rdd.toDF().select("salary").show(1)

# Salary


1563
, 126/35 TA28, Phường Thới An, Quận 12, Quận 12, TP.HCM
, khu vực, Quế Võ, Bắc Ninh
, Sa Đéc, Đồng Tháp
, G-503, Tầng 5 The Manor Officetel, 89 Nguyễn Hữu Cảnh, Phường 22, Quận Bình Thạnh, Bình Thạnh, TP.HCM
, Số 391 - 393 Quốc Lộ 51, khu phố 1, Phường Long Bình Tân, Biên Hòa, Đồng Nai
, 27 Lương Trúc Đàm, phường Hiệp Tân, quận Tân Phú, Tp.Hồ Chí Minh, Tân Phú, TP.HCM
, Bộ Phận Tổ chức _đào tạo - Số 2, Nguyễn Văn Thủ, Phường Đakao, Quận 1, TP.HCM
, Số 19, đường 07, KCN đô thị và dịch vụ VSIP, Bắc Ninh, Bắc Ninh, Bắc Ninh
, 24C Phan Đăng Lưu, Phường 6, Bình Thạnh, Bình Thạnh, TP.HCM
, 19 Đường 3A, KDC Vĩnh Lộc, P.Bình Hưng Hòa, Bình Tân, TP.HCM
, 99/23 Nguyễn Cửu Đàm, Phường Tân Sơn Nhì, Tân Phú, TP.HCM
,  618/34 Âu Cơ, phường 10,, Tân Bình, TP.HCM
, 56 Nguyễn Thanh Tuyền, Phường 2, Tân Bình, TP.HCM
, 12 Đường số 1, Khu phố 4, Phường Hiệp Bình Chánh, Thành phố Thủ Đức, Thủ Đức, TP.HCM
, Số 40/14 Khu phố Hòa Long, Phường Vĩnh Phú,, Thuận An, Bình Dương
, B1/6/15 Đoàn Nguyễn Tuân, Ấp

In [12]:
# Salary
def parse_salary(salary):
    if salary is None:
        return {"type": 0 }
    elif (salary.strip().lower() == "thoả thuận" or salary.strip().lower() == "thỏa thuận"):
        return {"type": 3 }
    elif "triệu" in salary:
        salary = salary.replace(" triệu", "").replace(",", ".").strip()
        if "-" in salary:
            min_salary, max_salary = salary.split("-")
            return {"type": 1, "min": float(min_salary), "max": float(max_salary) }
        else:
            return {"type": 2, "fixed_value": float(salary)}
# Age
def parse_age(age):
    if age is None:
        return { "type": 0 }
    age = age.strip().lower()
    if "tuổi" in age:
        age = age.replace(" tuổi", "").strip()
        if "-" in age:
            min_age, max_age = age.split("-")
            return { "type": 1, "min": int(min_age), "max": int(max_age) }
        else:
            return { "type": 2, "value": int(age) }
    return { "type": 0 }

# Experience
def parse_experience(experience):
    if experience is None or experience == "Chưa có kinh nghiệm":
        return { "type": 0 }
    if "Dưới" in experience:
        strip_experience = experience.replace("Dưới", "").replace("năm", "").strip()
        max = strip_experience
        return { "type": 3, "max": int(max) }
    elif "Hơn" in experience:
        strip_experience = experience.replace("Hơn", "").replace("năm", "").strip()
        min = strip_experience
        return { "type": 4, "min": int(min) }
    else:
        fixed = experience.replace("năm", "").strip()
        return { "type": 2, "fixed": int(fixed)}
    return { "type": 0 }

def parse_category(categories):
    if categories is None or len(categories) == 0:
        return []
    else :
        category_arr = []
        for category in categories:
            if category == 'Hành chính - Thư ký':
                category_arr.append('hc_ql')
            elif category == 'Giáo dục - Đào tạo':
                category_arr.append('gd_dt')
            elif category == 'Nghề nghiệp khác':
                category_arr.append('other')
            elif category == 'Quản lý tiêu chuẩn và chất lượng' or category == 'Quản lý dự án':
                category_arr.append('hc_ql')
            elif category == 'Thực tập sinh':
                category_arr.append('other')
            elif category == 'Nông - Lâm - Ngư nghiệp':
                category_arr.append('nn_ln_ts')
            elif category == 'An toàn lao động' or category == 'An ninh - Bảo vệ':
                category_arr.append('at_an')
            elif category == 'Vận Tải - Lái xe - Giao nhận':
                category_arr.append('vt')
            elif category == 'Bán sỉ - Bán lẻ - Quản lý cửa hàng':
                category_arr.append('kd')
            elif category == 'Hóa học - Hóa sinh':
                category_arr.append('kh_kt')
            elif category == 'IT Phần mềm':
                category_arr.append('it')
            elif category == 'Tài chính - Đầu tư - Chứng Khoán' or category == 'Ngân hàng':
                category_arr.append('tc_kt')
            elif category == 'Phân tích - Thống kê dữ liệu':
                category_arr.append('pt_tk')
            elif category == 'Kiến trúc - Thiết kế nội ngoại thất':
                category_arr.append('tk_kt_nt')
            elif category == 'Điện - Điện tử - Điện lạnh':
                category_arr.append('cn_sx')
            elif category == 'Thực phẩm - Đồ uống':
                category_arr.append('tp')
            elif category == 'Thiết kế - Sáng tạo nghệ thuật':
                category_arr.append('tk_kt_nt')
            elif category == 'Sản xuất - Lắp ráp - Chế biến':
                category_arr.append('cn_sx')
            elif category == 'Kế toán':
                category_arr.append('tc_kt')
            elif category == 'Bất động sản':
                category_arr.append('bds')
            elif category == 'Bán hàng - Kinh doanh':
                category_arr.append('kd')
            elif category == 'Khoa học - Kỹ thuật':
                category_arr.append('kh_kt')
            elif category == 'Thông tin - Truyền thông - Quảng cáo' or category == 'Marketing':
                category_arr.append('tt_mkt')
            elif category == 'Dầu khí':
                category_arr.append('cn_sx')
            elif category == 'Khách sạn - Nhà hàng - Du lịch':
                category_arr.append('dv')
            elif category == 'Truyền hình - Báo chí - Biên tập':
                category_arr.append('tt_mkt')
            elif category == 'Công nghệ thực phẩm - Dinh dưỡng':
                category_arr.append('tp')
            elif category == 'Chăn nuôi - Thú y':
                category_arr.append('nn_ln_ts')
            elif category == 'Xuất Nhập Khẩu':
                category_arr.append('kd')
            elif category == 'Vận hành - Bảo trì - Bảo dưỡng':
                category_arr.append('cn_sx')
            elif category == 'Dược phẩm':
                category_arr.append('yt_sk')
            elif category == 'Nhân sự':
                category_arr.append('hc_ql')
            elif category == 'Xây dựng':
                category_arr.append('xd')
            elif category == 'Cơ khí - Ô tô - Tự động hóa':
                category_arr.append('cn_sx')
            elif category == 'Xuất bản - In ấn':
                category_arr.append('xb')
            elif category == 'Kiểm toán':
                category_arr.append('tc_kt')
            elif category == 'Môi trường - Xử lý chất thải':
                category_arr.append('cn_sx')
            elif category == 'IT Phần cứng - Mạng':
                category_arr.append('it')
            elif category == 'Khai thác năng lượng - Khoáng sản - Địa chất':
                category_arr.append('cn_sx')
            elif category == 'Biên phiên dịch':
                category_arr.append('law')
            elif category == 'Bảo hiểm':
                category_arr.append('law')
            elif category == 'Dệt may - Da giày - Thời trang':
                category_arr.append('tk_kt_nt')
            elif category == 'Bưu chính viễn thông':
                category_arr.append('dv')
            elif category == 'Chăm sóc khách hàng':
                category_arr.append('dv')
            elif category == 'Lao động phổ thông':
                category_arr.append('ldpt')
            elif category == 'Luật - Pháp Lý - Tuân thủ':
                category_arr.append('law')
            elif category == 'Thu mua - Kho Vận - Chuỗi cung ứng':
                category_arr.append('kd')
            elif category == 'Y tế - Chăm sóc sức khỏe':
                category_arr.append('yt_sk')
            else:
                print("None value match job category" + category)
        return category_arr
        
# Type
df = df.withColumn(
    "type",
     when(col("type") == "Toàn thời gian cố định", array(lit(0)))
    .when(col("type") == "Bán thời gian cố định", array(lit(2)))
    .when(col("type") == "Toàn thời gian tạm thời", array(lit(1)))
    .when(col("type") == "Khác", array(lit(5)))
    .otherwise(array(lit(0)))
)

# Certificate 
# Type
df = df.withColumn(
    "certificate",
     when(col("certificate") == None, 0)
    .when(col("certificate") == "Chứng chỉ", 1)
    .when(col("certificate") == "Trung học", 2)
    .when(col("certificate") == "Trung cấp", 3)
    .when(col("certificate") == "Cao đẳng", 4)
    .when(col("certificate") == "Đại học", 5)
    .when(col("certificate") == "Trên đại học", 6)
    .otherwise(0)
)

# Sex
df = df.withColumn(
    "sex",
     when(col("sex") == "Nam", 1)
    .when(col("sex") == "Nữ", 2)
    .otherwise(0)
)

# Expiration
df = df.withColumn("expiration", to_timestamp(concat(col("expiration"), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"))

# Update at
df = df.withColumn("update_time", to_timestamp(concat(col("update_time"), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"))

# Description
def clean_description(description):
    return [re.sub(r'^[\-\•\d\.\s]+', '', desc).strip() for desc in description]
clean_description_udf = udf(clean_description, ArrayType(StringType()))
df = df.withColumn("description", clean_description_udf(col("description")))

# Role
def classify_role(role):
    if role == "Chuyên viên- nhân viên":
        return 0
    elif role == "Quản lý cấp cao" or role == "Quản lý cấp trung":
        return 1
    elif role == "Cộng tác viên":
        return 6
    elif role == "Quản lý nhóm- giám sát":
        return 5
    elif role == "Chuyên gia":
        return 7
    else:
        return 0
classify_role_udf = udf(classify_role, IntegerType())
df = df.withColumn("role", classify_role_udf(col("role")))

def normalize_benefit(benefits):
    return list(map(lambda benefit: re.sub(r'^[*\-+]+', '', benefit).strip(), benefits))

benefit_udf = udf(normalize_benefit, ArrayType(StringType()))
df = df.withColumn("benefit", benefit_udf(col("benefit")))
df = df.withColumn("requirement", benefit_udf(col("requirement")))

# location
def normalize_location(locations):
    arr = []
    for location in locations:
        parts = [part.strip() for part in location.split(',') if part.strip()]
        if len(parts) >= 2:
            province = parts[-1]
            district = parts[-2]
            address = ', '.join(parts[:-2])
            arr.append({"province": province, "district": district, "address": address})
        elif len(parts) == 1:
            arr.append ({"province": parts[-1], "district": None, "address": None })
        else: 
            arr.append({"address": None, "district": None, "province": None})
    return arr
    
rdd = df.rdd.map(lambda row: {
    **row.asDict(),
    "salary": parse_salary(row["salary"]),
    "age": parse_age(row["age"]),
    "location": normalize_location(row["location"]),
    "experience": parse_experience(row["experience"]),
    "field": parse_category(row["category"])
})


def insert_into_mongodb(partition):
    client = pymongo.MongoClient("mongodb://localhost:27017/", username='admin', password='20194856')
    db = client["thesis"]
    collection = db["jobs"]

    for record in partition:
        collection.insert_one(record)

# Insert dữ liệu vào collection "jobs"
rdd.foreachPartition(insert_into_mongodb)